In [1]:
import sys
import time
from ortools.sat.python import cp_model

In [5]:
# https://developers.google.com/optimization/scheduling/employee_scheduling
# num_nurses = 4
# num_shifts = 3
# num_days = 3
X = 16
num_nurses = X*X
num_shifts = X
num_days = X
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

model = cp_model.CpModel()

shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

# Each shift is assigned to exactly one nurse in the schedule period.
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)  

# Each nurse works at most one shift per day.    
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_nurse <= sum(shifts_worked))
    model.Add(sum(shifts_worked) <= max_shifts_per_nurse)

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = False

class SolPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        if self._solution_count>0: return
        self._solution_count += 1
        print('Solution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % d)
            workings = []
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        workings.append((n, s))
                        # print('  Nurse %i works shift %i' % (n, s))
                # if not is_working:
                #     print('  Nurse {} does not work'.format(n))
            print(f"""  {[f'{n}:{s}' for n, s in workings]}""")
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 1
solution_printer = SolPrinter(shifts, num_nurses,
                                                num_days, num_shifts,
                                                solution_limit)

solver.Solve(model, solution_printer)
print('  - wall time      : %f s' % solver.WallTime())

Solution 1
Day 0
  ['0:0', '2:2', '3:3', '4:4', '5:5', '6:6', '7:7', '8:8', '9:9', '10:10', '11:11', '12:12', '13:13', '14:14', '15:15', '255:1']
Day 1
  ['16:0', '17:1', '18:2', '19:3', '20:4', '21:5', '22:6', '23:7', '24:8', '25:9', '26:10', '27:11', '28:12', '29:13', '30:14', '31:15']
Day 2
  ['32:0', '33:1', '34:2', '35:3', '36:4', '37:5', '38:6', '39:7', '40:8', '41:9', '42:10', '43:11', '44:12', '45:13', '46:14', '47:15']
Day 3
  ['48:0', '49:1', '50:2', '51:3', '52:4', '53:5', '54:6', '55:7', '56:8', '57:9', '58:10', '59:11', '60:12', '61:13', '62:14', '63:15']
Day 4
  ['64:0', '65:1', '66:2', '67:3', '68:4', '69:5', '70:6', '71:7', '72:8', '73:9', '74:10', '75:11', '76:12', '77:13', '78:14', '79:15']
Day 5
  ['80:0', '81:1', '82:2', '83:3', '84:4', '85:5', '86:6', '87:7', '88:8', '89:9', '90:10', '91:11', '92:12', '93:13', '94:14', '95:15']
Day 6
  ['96:0', '97:1', '98:2', '99:3', '100:4', '101:5', '102:6', '103:7', '104:8', '105:9', '106:10', '107:11', '108:12', '109:13', '110

In [ ]:
model = cp_model.CpModel()

# board_size = 256
board_size = 128
# There are `board_size` number of variables, one for a queen in each column
# of the board. The value of each variable is the row that the queen is in.
queens = [
    model.NewIntVar(0, board_size - 1, 'x%i' % i) for i in range(board_size)
]

# All rows must be different.
model.AddAllDifferent(queens)

# No two queens can be on the same diagonal.
model.AddAllDifferent(queens[i] + i for i in range(board_size))
model.AddAllDifferent(queens[i] - i for i in range(board_size))

class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        if self.__solution_count>1: return
        self.__solution_count += 1
        current_time = time.time()
        print('Solution %i, time = %f s' %
              (self.__solution_count, current_time - self.__start_time))

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i.
                    print('Q', end=' ')
                else:
                    print('_', end=' ')
            print()
        print()
        self.StopSearch()

solver = cp_model.CpSolver()
solution_printer = NQueenSolutionPrinter(queens)
solver.parameters.enumerate_all_solutions = False
solver.Solve(model, solution_printer)

Solution 1, time = 1.268924 s
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Q _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Q _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Q _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Q _ _ _ _ _ _ _ _ _ _ _ _ _ _

4